# Generación de Predicciones con Regresión Lineal
Este notebook implementa una regresión lineal para predecir valores en un conjunto de datos simulados.

## 1. Importación de Bibliotecas

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


## 2. Extracción y limpieza de datos

<span style="color:white; font-weight:bold">2.1 Extracción</span>

Los datos se generaron directamente de la base de datos MSSQL con las ventas reales de la empresa HBS-Delli, del 2020 a la fecha (diciembre 2024).

In [12]:
df = pd.read_excel("../data/Ventas_CapstoneProject_Agrupado.xlsx", sheet_name="Ventas")

**2.2 funciones**

In [9]:
def eliminar_datos_atipicos(input_df):
    
    # Calcular Q1 (percentil 25) y Q3 (percentil 75)
    Q1 = input_df['cantidad'].quantile(0.25)
    Q3 = input_df['cantidad'].quantile(0.75)
    IQR = Q3 - Q1

    # Definir límites para valores no atípicos
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filtrar el DataFrame eliminando valores atípicos
    df_sin_picos = input_df[(input_df['cantidad'] >= lower_bound) & (input_df['cantidad'] <= upper_bound)]

    #Comentar si se quieren utilizar datos atipicos
    # df = df_sin_picos

    return df_sin_picos

<span style="color:white; font-weight:bold">2.2 Limpieza</span>

La mayor parte de la limpieza de datos se realizaron directamente en la base de datos MSSQL, sin embargo se utilizo pandas para realizar una validación de la información y preparar el dataset para su analisis.

In [10]:
#validar nulos
if df.isnull().sum().sum() > 0:
    df = df.dropna()
    print("Se eliminaron las filas con valores nulos.")
else:
    print("No se encontraron valores nulos en el DataFrame.")

# Preparando información de la venta
df = df.rename(columns={'FechaEmision': 'fecha_emision'
                        , "Empresa": "empresa"
                        , "Semana_Anio": "semana_anio"
                        })

# Preparando informacion del artículo
df['numero_articulo'] = pd.to_numeric(
                                    df['Articulo'].str.extract(r'(\d+)')[0]     # Extraer solo números
                                    ,errors='coerce'                            # Convertir valores inválidos a NaN
                                    ).fillna(0).astype(int)                     # Reemplazar NaN por 0 y convertir a entero
df = df.rename(columns={"Articulo": "articulo"
                        ,'Desc_Articulo': 'desc_articulo'
                        , 'FechaEmision': 'fecha_emision'
                        , "Empresa": "empresa"
                        , "Mov": "mov"
                        , "MovId": "movId"
                        , "Lote": "lote"
                        })

# Preparando información del cliente
df = df.rename(columns={'Numero_Cliente': 'numero_cliente'
                        , 'Nombre_Cliente': 'nombre_cliente'
                        , "Rama": "rama_cliente"
                        , "Tipo" : "tipo_cliente"
                        , "Linea": "linea"
                        , "Modulo": "modulo"
                        })
df['numero_cliente'] = df['numero_cliente'].fillna('Desconocido')

# Preparando información de las cantidad
df = df.rename(columns={"Cantidad": "cantidad"
                        , "Lote": "lote"
                        , "UM": "unidad_medida"
                        , "Precio": "precio"
                        , "SubTotal": "subTotal"
                        , "DescuentoLineal": "descuento_lineal"
                        , "IVA": "iva"
                        , "IEPS": "ieps"
                        , "ImporteTotal": "importe_total"
                        , "Costo_Unitario": "costo_unitario"
                        , "Costo_Total": "costo_total"
                          })
# validando columna de tipo fecha
if (df["fecha_emision"].dtypes=="object"):
    df["fecha_emision"] = pd.to_datetime(df["fecha_emision"], errors='coerce', format='%d/%m/%Y')
    print(f"La columna '{"fecha_emision"}' ha sido convertida a datetime.")
else:
    print(f"La columna '{"fecha_emision"}' ya es de tipo datetime, no es necesario convertir.")

# Eliminando información no requerida para el analisis
df = df.drop(columns=["IEPS_Cuota", "Numero_Orden"])

df = eliminar_datos_atipicos(df)

No se encontraron valores nulos en el DataFrame.
La columna 'fecha_emision' ya es de tipo datetime, no es necesario convertir.


In [ ]:
df.info()

<h2>Split historical and future data</h2>

In [11]:
# Step 1: Data preparation
ventas_data = df.copy()
# Selecting relevant features and target variable
features = ventas_data.drop(columns=['Cantidad', 'FechaEmision', 'Numero_Orden', 'Nombre_Cliente', 'Desc_Articulo'])
target = ventas_data['Cantidad']

# Convert categorical features to numerical using one-hot encoding
features_encoded = pd.get_dummies(features, drop_first=True)

# Handle missing values if any
features_encoded.fillna(0, inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# Step 2: Model selection and training
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Step 3: Predictions and evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, mae, r2


KeyError: "['Cantidad', 'FechaEmision', 'Numero_Orden', 'Nombre_Cliente', 'Desc_Articulo'] not found in axis"